In [3]:
from llama_index.legacy.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.legacy.llms import OpenAI
import os
import json
from llama_index.legacy.schema import TextNode
from llama_index.legacy.node_parser import SentenceWindowNodeParser, SemanticSplitterNodeParser
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.legacy.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
) 

## load evv variables
HF_KEY             = os.environ["HUGGINGFACE_API_KEY"]
dense_embedder_api = os.environ["HF_API_URL"]
OPENAI_KEY         = os.environ["OPENAI_API_KEY"]
GROQ_API_KEY       = os.environ["GROQ_API_KEY"]